In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.1.8
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data <- read.csv("selected.csv")
length(data[,"Labels"])
data <- data[sample(1:length(data$Labels)), ] #random shuffle
length(data[,"Labels"])
val_data <- data[as.integer(0.9 * length(data$Labels)):length(data$Labels),]
data <- data[1:as.integer(0.9 * length(data$Labels)),]
length(data[,"Labels"])
length(val_data[,"Labels"])

[1] 10240

[1] 10240

[1] 9216

[1] 1025

In [3]:
pc <- c(0,0,0,0,0,0)
for(i in 0:5) {
    pc[i+1] <- length(data[data[, "Labels"]==i, "Labels"])
}

In [4]:
full_text <- paste(data$Text, sep=" ", collapse=" ")
#tokens <- c()
#for(t in data$Text) {
#    tokens <- c(tokens, strsplit(t, " ")[[1]])
#}
#tokens <- unique(tokens)
tokens <- unique(strsplit(full_text, " ")[[1]])
tokens <- tokens[- (tokens=="")]

In [5]:
## multinomial model
ptc <- data.frame(
    token = tokens, 
    ct0 = 0,
    ct1 = 0,
    ct2 = 0,
    ct3 = 0,
    ct4 = 0,
    ct5 = 0,
    #ctp0 = 0, 
    #ctp1 = 0, 
    #ctp2 = 0, 
    #ctp3 = 0, 
    #ctp4 = 0, 
    #ctp5 = 0, 
    pt0 = 0,
    pt1 = 0,
    pt2 = 0,
    pt3 = 0,
    pt4 = 0,
    pt5 = 0
)



for(i in 0:5) {
    class_text <- strsplit(paste(data[data[, "Labels"]==i, "Text"], sep=" ", collapse=" "), " ")[[1]]
    
    for(j in 1:length(tokens)) {
        ptc[j,paste("ct",i, sep="")] <- sum(class_text==ptc[j, "token"]) + 1
        #ptc[j,paste("ctp",i, sep="")] <- length(tokens) + length(class_text) - ptc[j,paste("ct",i, sep="")]
    }
    #ptc[,paste("pt",i, sep="")] <- ptc[,paste("ct",i, sep="")] / ptc[,paste("ctp",i, sep="")]
    ptc[,paste("pt",i, sep="")] <- ptc[,paste("ct",i, sep="")] / (length(class_text) + length(tokens))
}
ptc <- ptc[,c("token","pt0", "pt1", "pt2", "pt3", "pt4", "pt5")]

In [6]:
## bernoulli model
ptc_b <- data.frame(
    token = tokens, 
    ct0 = 0,
    ct1 = 0,
    ct2 = 0,
    ct3 = 0,
    ct4 = 0,
    ct5 = 0,
    #ctp0 = 0, 
    #ctp1 = 0, 
    #ctp2 = 0, 
    #ctp3 = 0, 
    #ctp4 = 0, 
    #ctp5 = 0, 
    pt0 = 0,
    pt1 = 0,
    pt2 = 0,
    pt3 = 0,
    pt4 = 0,
    pt5 = 0
)


#print(c(paste(" ", ptc_b$token, " ", sep="")))

for(i in 0:5) {
    count <- 0
    l <- length(data[data[, "Labels"]==i, "Text"])
    sels <- c()
    for(t in data[data[, "Labels"]==i, "Text"]) {
        #print(str_extract(t, paste(" ", ptc_b$token, " ", sep="")))
        #print(t)
        #print(sels)
        sels <- c(sels, str_extract(t, paste(" ", ptc_b$token, " ", sep="")))
    }   
    sels <- sels[!is.na(sels)]
    sels <- gsub(" ", "", sels, fixed=T)
    #print(sels[1:100])
    #print(ptc_b[1:100,]$token)
    for(j in 1:length(ptc_b$token)) {
        ptc_b[j,paste("ct",i, sep="")] <- sum(sels==ptc_b[j, "token"])+1
    }
    ptc_b[,paste("pt",i, sep="")] <- ptc_b[,paste("ct",i, sep="")] / (length(tokens) + l)
}
ptc_b <- ptc_b[,c("token","pt0", "pt1", "pt2", "pt3", "pt4", "pt5")]

In [7]:
#write.csv(ptc, "prob_token_by_class.csv")
#write.csv(ptc_b, "prob_token_by_class_b.csv")
# save for later

In [8]:
right <- 0
right_b <- 0
for(k in 1:length(val_data[,"Labels"])) {
    vec <- strsplit(val_data[k, "Text"], " ")[[1]]
    probs <- log(pc)
    probs_b <- log(pc)
    
    #print(probs)
    #print(probs_b)
    #flush.console()
    
    for(h in vec) {
        for(i in 0:5) {
            #print(ptc[ptc[,"token"]==h ,paste("pt",i, sep="")])
            inc <- log(ptc[ptc[,"token"]==h ,paste("pt",i, sep="")])
            if ( length(inc) == 1 && ! any(is.na(inc)) ) {
                probs[i+1] <- probs[i+1] + inc
            }
            
            inc_b <- log(ptc_b[ptc_b[,"token"]==h ,paste("pt",i, sep="")])
            if ( length(inc_b) == 1 && ! any(is.na(inc_b)) ) {
                probs_b[i+1] <- probs_b[i+1] + inc_b 
            }
            
        }
    }
    
    #print(probs)
    #print(probs_b)
    #flush.console()
    
    if( (0:5)[probs == max(probs)] == val_data[k, "Labels"]) {
        right <- right + 1
    }
    if( (0:5)[probs_b == max(probs_b)] == val_data[k, "Labels"]) {
        right_b <- right_b + 1
    }
}
print(100*right/length(val_data[,"Labels"]))
print(100*right_b/length(val_data[,"Labels"]))

[1] 24.78049
[1] 24.97561
